# 🔖 Tagging
Tagging refers to the process of assigning labels or tags to parts of text to classify or annotate them. This is typically done to prepare text for downstream processing or understanding.

In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

In [10]:
import os
import openai
from dotenv import load_dotenv
from typing import List, Optional
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

from pydantic import BaseModel, Field 

from langchain.document_loaders import WebBaseLoader

In [11]:
model = ChatOpenAI(temperature=0)

In [12]:
class Tagging(BaseModel):
    """Tag the piece of text with sentiment and language."""
    sentiment: str = Field(description="sentiment of text: pos, neg, or neutral")
    language: str = Field(description="language (ISO 639-1 code like 'en', 'fr', 'it')")



In [15]:
tagging_fn = [convert_pydantic_to_openai_function(Tagging)]

tagger_model = model.bind(functions=tagging_fn, function_call={"name": "Tagging"})

In [16]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Tag the input by detecting sentiment and language."),
    ("user", "{input}")
])


In [17]:
tagging_chain = prompt | tagger_model | JsonOutputFunctionsParser() 

In [20]:
tagging_chain.invoke(input="I hate langchain framework ")

{'sentiment': 'neg', 'language': 'en'}

## 📥 Extraction
Extraction means pulling specific structured information from unstructured or semi-structured text. This helps in converting free-form language into a machine-usable format.

In [21]:
class Person(BaseModel):
    name: str
    age: Optional[int] = None

In [23]:
class Information(BaseModel):
    people: List[Person]

extraction_fn = [convert_pydantic_to_openai_function(Information)]
extraction_model = model.bind(functions=extraction_fn, function_call={"name": "Information"})

In [24]:

extract_prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract names and ages of people from the input text."),
    ("user", "{input}")
])

In [25]:
extraction_chain = extract_prompt | extraction_model | JsonOutputFunctionsParser()

In [26]:
extraction_chain.invoke(input="I have two friends, John and Mary. John is 25 years old and Mary is 30 years old.")

{'people': [{'name': 'John', 'age': 25}, {'name': 'Mary', 'age': 30}]}

# exmaple  webbased for tagging and extraction 

In [27]:
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")

In [28]:
documents = loader.load()
page_content = documents[0].page_content[:10000]

In [29]:
class Overview(BaseModel):
    summary: str
    language: str
    keywords: str

In [30]:
overview_fn = [convert_pydantic_to_openai_function(Overview)]

In [31]:
overview_fn

[{'name': 'Overview',
  'description': '',
  'parameters': {'properties': {'summary': {'type': 'string'},
    'language': {'type': 'string'},
    'keywords': {'type': 'string'}},
   'required': ['summary', 'language', 'keywords'],
   'type': 'object'}}]

In [32]:
overview_model = model.bind(functions=overview_fn, function_call={"name": "Overview"})

In [33]:
overview_prompt = ChatPromptTemplate.from_messages([
    ("system", "Summarize the article and extract keywords and language."),
    ("user", "{input}")
])

In [34]:
overview_chain = overview_prompt | overview_model | JsonOutputFunctionsParser() 

In [35]:
print(overview_chain.invoke({"input": page_content}))

{'summary': 'The article discusses the concept of building autonomous agents powered by LLM (large language model) as the core controller. It explores components like planning, memory, and tool use in these agents, along with techniques such as task decomposition and self-reflection. Various case studies and proof-of-concept examples are presented to showcase the potential of LLM-powered agents as problem solvers.', 'language': 'English', 'keywords': 'LLM, autonomous agents, planning, memory, tool use, task decomposition, self-reflection, proof-of-concept, case studies'}


In [36]:
# Extract Papers Mentioned
class Paper(BaseModel):
    title: str
    author: Optional[str] = None

In [37]:
class Info(BaseModel):
    papers: List[Paper]

In [38]:

papers_fn = [convert_pydantic_to_openai_function(Info)]

In [39]:
papers_fn

[{'name': 'Info',
  'description': '',
  'parameters': {'properties': {'papers': {'items': {'properties': {'title': {'type': 'string'},
       'author': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
        'default': None}},
      'required': ['title'],
      'type': 'object'},
     'type': 'array'}},
   'required': ['papers'],
   'type': 'object'}}]

In [40]:
paper_model = model.bind(functions=papers_fn, function_call={"name": "Info"})


paper_prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract titles and authors of papers mentioned in the text."),
    ("user", "{input}")
])

In [43]:
paper_chain = paper_prompt | paper_model | JsonOutputFunctionsParser()
paper_chain.invoke({"input": page_content})


{'papers': [{'title': 'Chain of Thought', 'author': 'Wei et al. 2022'},
  {'title': 'Tree of Thoughts', 'author': 'Yao et al. 2023'},
  {'title': 'LLM+P', 'author': 'Liu et al. 2023'},
  {'title': 'ReAct', 'author': 'Yao et al. 2023'},
  {'title': 'Reflexion', 'author': 'Shinn & Labash 2023'},
  {'title': 'Chain of Hindsight', 'author': 'Liu et al. 2023'},
  {'title': 'Algorithm Distillation', 'author': 'Laskin et al. 2023'}]}

# routing

## 
 Routing means determining which tool (function) should be executed based on the user's input.

In [52]:
from langchain.schema.agent import AgentFinish
from langchain.agents import tool
import requests
import wikipedia
import json
from langchain.tools.render import format_tool_to_openai_function 
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [ ]:
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location")
    longitude: float = Field(..., description="Longitude of the location")



In [49]:
@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> str:
    """Fetch current temperature for a location using Open-Meteo API."""
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m",
        "forecast_days": 1,
    }
    response = requests.get(BASE_URL, params=params)
    if response.status_code != 200:
        return "Weather API error."
    data = response.json()
    current_time = data["hourly"]["time"][0]
    current_temp = data["hourly"]["temperature_2m"][0]
    return f"At {current_time}, it is {current_temp}°C."

In [50]:
@tool 
def search_wikipedia(query:str )-> str:
    """Return Wikipedia summaries for a given query."""
    pages = wikipedia.search(query)
    summaries = []
    for title in pages[:3]:
        try:
            page = wikipedia.page(title, auto_suggest=False)
            summaries.append(f"**{title}**\n{page.summary}")
        except:
            pass
    return "\n\n".join(summaries) if summaries else "No good results found."


In [53]:
tools = [get_current_temperature, search_wikipedia]
openai_tools = [format_tool_to_openai_function(t) for t in tools]



/var/folders/nn/3vv9s5d557bc1lhnccljp37w0000gn/T/ipykernel_88885/2175230857.py:2: LangChainDeprecationWarning: The function `_format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  openai_tools = [format_tool_to_openai_function(t) for t in tools]


In [54]:
model = ChatOpenAI(temperature=0).bind(functions=openai_tools)

In [55]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful but sassy assistant."),
    ("user", "{input}")
])

In [56]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [60]:
def route (result):
    if isinstance(result,AgentFinish):
        return result.return_values["output"]
    tool_map = {
        "search_wikipedia": search_wikipedia,
        "get_current_temperature": get_current_temperature

    }
    return tool_map[result.tool].run(result.tool_input)

router_chain= chain | route 

In [62]:
print("\n✅ Weather tool:")
print(router_chain.invoke({"input": "What is the weather in Bangalore?"}))


✅ Weather tool:
At 2025-04-23T00:00, it is 22.2°C.


In [63]:
print("\n✅ Wikipedia  tool:")
print(router_chain.invoke({"input": "Who is Naredendra modi?"}))


✅ Wikipedia  tool:
**Narendra Modi**
Narendra Damodardas Modi (born 17 September 1950) is an Indian politician who has served as the  prime minister of India since 2014. Modi was the chief minister of Gujarat from 2001 to 2014 and is the member of parliament (MP) for Varanasi. He is a member of the Bharatiya Janata Party (BJP) and of the Rashtriya Swayamsevak Sangh (RSS), a far-right Hindu nationalist paramilitary volunteer organisation. He is the longest-serving prime minister outside the Indian National Congress.
Modi was born and raised in Vadnagar in northeastern Gujarat, where he completed his secondary education. He was introduced to the RSS at the age of eight. At the age of 18, he was married to Jashodaben Modi, whom he abandoned soon after, only publicly acknowledging her four decades later when legally required to do so. Modi became a full-time worker for the RSS in Gujarat in 1971. The RSS assigned him to the BJP in 1985 and he rose through the party hierarchy, becoming gen